<a href="https://colab.research.google.com/github/vrra/FGAN-Build-a-thon/blob/main/spacy-example-triplets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
pip install -U spacy

In [15]:
pip install textacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.9 MB/s eta 0:00:00


In [17]:
import spacy


In [58]:
import textacy


In [60]:
import textacy.extract

In [61]:
nlp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [62]:
text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with Recode earlier "
        "this week.")

In [63]:
doc = nlp(text)


In [64]:
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

Noun phrases: ['Sebastian Thrun', 'self-driving cars', 'Google', 'few people', 'the company', 'him', 'I', 'you', 'very senior CEOs', 'major American car companies', 'my hand', 'I', 'Thrun', 'an interview', 'Recode']
Verbs: ['start', 'work', 'drive', 'take', 'tell', 'shake', 'turn', 'talk', 'say']


In [65]:
# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

Sebastian Thrun PERSON
Google ORG
2007 DATE
American NORP
Thrun PERSON
Recode ORG
earlier this week DATE


In [160]:
sentences = ["All living things are made of cells.",
             "Aaron is a good boy.",
             "cells has organelles",
             "Kennedy is in UK."]

In [136]:
verb_patterns = [[{"POS":"AUX"}, {"POS":"VERB"},
                  {"POS":"ADP"}],
                 [{"POS":"AUX"}]]

In [137]:

def contains_root(verb_phrase, root):
    vp_start = verb_phrase.start
    vp_end = verb_phrase.end
    if (root.i >= vp_start and root.i <= vp_end):
        return True
    else:
        return False

In [138]:
def find_root_of_sentence(doc):
    root_token = None
    for token in doc:
        if (token.dep_ == "ROOT"):
            root_token = token
    return root_token

In [139]:
def get_verb_phrases(doc):
    root = find_root_of_sentence(doc)
    verb_phrases= textacy.extract.matches.token_matches(doc,verb_patterns)

    new_vps = []
    for verb_phrase in verb_phrases:
        if (contains_root(verb_phrase, root)):
            new_vps.append(verb_phrase)
    return new_vps

In [140]:
def longer_verb_phrase(verb_phrases):
    longest_length = 0
    longest_verb_phrase = None
    for verb_phrase in verb_phrases:
        if len(verb_phrase) > longest_length:
            longest_verb_phrase = verb_phrase
    return longest_verb_phrase

In [115]:
def find_noun_phrase(verb_phrase, noun_phrases, side):
    for noun_phrase in noun_phrases:
        if (side == "left" and \
            noun_phrase.start < verb_phrase.start):
            return noun_phrase
        elif (side == "right" and \
              noun_phrase.start > verb_phrase.start):
            return noun_phrase

In [158]:
def find_triplet(sentence):
    doc = nlp(sentence)
    verb_phrases = get_verb_phrases(doc)
    noun_phrases = doc.noun_chunks
    verb_phrase = None

    if (len(verb_phrases) > 1):
        verb_phrase = \
        longer_verb_phrase(list(verb_phrases))
    elif (len(verb_phrases)==1):
        verb_phrase = verb_phrases[0]
    elif (len(verb_phrases)==0):
        print("verb_phrases is NULL")
        return("","","")

    left_noun_phrase = find_noun_phrase(verb_phrase,
                                        noun_phrases,
                                        "left")
    right_noun_phrase = find_noun_phrase(verb_phrase,
                                         noun_phrases,
                                         "right")
    return (left_noun_phrase, verb_phrase,
            right_noun_phrase)

In [161]:
for sentence in sentences:
    (left_np, vp, right_np) = find_triplet(sentence)
    print("Triplets = ",left_np, "\t", vp, "\t", right_np)

Triplets =  All living things 	 are made of 	 cells
Triplets =  Aaron 	 is 	 a good boy
verb_phrases is NULL
Triplets =   	  	 
Triplets =  Kennedy 	 is 	 UK
